<a href="https://colab.research.google.com/github/Leticiapp/HandsOnSegfy/blob/main/HandsOnSegfy2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [173]:
#imports
import pandas as pd
import numpy as np
import missingno as ms
import requests as rq
from io import BytesIO

In [174]:
# configuração para não exibir os warnings
import warnings
warnings.filterwarnings("ignore")

In [175]:
#import para salvar arquivos .csv no drive
#from google.colab import drive

# drive.mount('/content/drive')

In [176]:
# URL de importação do arquivo cepbr_geo.csv
url = "https://raw.githubusercontent.com/Leticiapp/HandsOnSegfy/main/data/cepbr_geo.csv"

# Colunas
colunas = ['cep', 'latitude', 'longitude']

# Leitura do arquivo
dataset = pd.read_csv(url, names=colunas, skiprows=1, delimiter='|')

In [177]:
#verificando as primeiras linhas da importação
dataset.head()

,cep,latitude,longitude
0,1029901,-23.5427230,-46.6332583
1,1227200,-23.5475983,-46.6587135
2,1251000,-23.5443440,-46.6733857
3,1255010,-23.5474499,-46.6817876
4,1308040,-23.5279687,-46.3963275


In [178]:
#Verificando as dimensões do dataset (linhas x colunas)
dataset.shape

(4990, 3)

In [179]:
#Informações do dataset
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4990 entries, 0 to 4989
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   cep        4990 non-null   int64 
 1   latitude   4990 non-null   object
 2   longitude  4990 non-null   object
dtypes: int64(1), object(2)
memory usage: 117.1+ KB


In [180]:
#ordenação por cep
dataset = dataset.sort_values(by=['cep'])
dataset.head()

,cep,latitude,longitude
0,1029901,-23.5427230,-46.6332583
1,1227200,-23.5475983,-46.6587135
2,1251000,-23.5443440,-46.6733857
3,1255010,-23.5474499,-46.6817876
4,1308040,-23.5279687,-46.3963275


In [181]:
#Transformação dos campos latitude e longitude para String para possibilitar operações furutas
dataset['latitude'] = dataset['latitude'].str.replace(r'\D', '', regex=True)
dataset['longitude'] = dataset['longitude'].str.replace(r'\D', '', regex=True)

dataset['latitude'] = pd.to_numeric(dataset['latitude'])
dataset['longitude'] = pd.to_numeric(dataset['longitude'])

dataset.head()

,cep,latitude,longitude
0,1029901,235427230.0,466332583.0
1,1227200,235475983.0,466587135.0
2,1251000,235443440.0,466733857.0
3,1255010,235474499.0,466817876.0
4,1308040,235279687.0,463963275.0


In [182]:
#Adição de um novo campo "cep5" com apenas os 5 primeiros dígitos de cada cep
dataset['cep'] = dataset['cep'].astype(str)
dataset['cep5'] = dataset['cep'].str.slice(0,5)

#converte novamente para int
dataset['cep'] = dataset['cep'].astype(int)
dataset['cep5'] = dataset['cep5'].astype(int)

dataset.head()

,cep,latitude,longitude,cep5
0,1029901,235427230.0,466332583.0,10299
1,1227200,235475983.0,466587135.0,12272
2,1251000,235443440.0,466733857.0,12510
3,1255010,235474499.0,466817876.0,12550
4,1308040,235279687.0,463963275.0,13080


In [183]:
#criação de um novo dataset contendo todos os ceps entre 01000 a 99999

myRange = np.arange(1000,100000,1)
df_ceps = pd.DataFrame({"cep": myRange})
df_ceps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99000 entries, 0 to 98999
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   cep     99000 non-null  int64
dtypes: int64(1)
memory usage: 773.6 KB


In [184]:
df_ceps.head()

,cep
0,1000
1,1001
2,1002
3,1003
4,1004


In [185]:
df_ceps.tail()

,cep
98995,99995
98996,99996
98997,99997
98998,99998
98999,99999


In [186]:
#Criação de um novo campo 'closest' no dataset de ceps, o qual será preenchido com os ceps mais próximos localizados na base de dados cepbr_geo
df_ceps['closest'] = ''
df_ceps.head()

,cep,closest
0,1000,
1,1001,
2,1002,
3,1003,
4,1004,


In [187]:
#procura os dois ceps mais próximos que estão presentes na base cepbr_geo para os 5 primeiros dígitos cada cep
for i, v in enumerate(df_ceps['cep'].value_counts()):
  df_sort = dataset.iloc[(dataset['cep5']- df_ceps['cep'][i]).abs().argsort()[:2]]
  df_ceps['closest'][i] = df_sort['cep5'].tolist()



In [188]:
#print do campo closest preenchido
pd.set_option('display.max_colwidth', None)
df_ceps.head(10)

,cep,closest
0,1000,"[10299, 11010]"
1,1001,"[10299, 11010]"
2,1002,"[10299, 11010]"
3,1003,"[10299, 11010]"
4,1004,"[10299, 11010]"
5,1005,"[10299, 11010]"
6,1006,"[10299, 11010]"
7,1007,"[10299, 11010]"
8,1008,"[10299, 11010]"
9,1009,"[10299, 11010]"


In [189]:
df_ceps.tail(10)

,cep,closest
98990,99990,"[99901, 99811]"
98991,99991,"[99901, 99811]"
98992,99992,"[99901, 99811]"
98993,99993,"[99901, 99811]"
98994,99994,"[99901, 99811]"
98995,99995,"[99901, 99811]"
98996,99996,"[99901, 99811]"
98997,99997,"[99901, 99811]"
98998,99998,"[99901, 99811]"
98999,99999,"[99901, 99811]"


In [190]:
#transformação do campo closest para o tipo de dado string
df_ceps['closest'] = df_ceps['closest'].astype(pd.StringDtype())

In [191]:
#informações do dataset
df_ceps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99000 entries, 0 to 98999
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   cep      99000 non-null  int64 
 1   closest  99000 non-null  string
dtypes: int64(1), string(1)
memory usage: 1.5 MB


In [192]:
#Criação de dois novos campos closest1 e closest2, separando os dois ceps em diferentes campos e realizando os tratamentos necessários
df_ceps['closest'] = df_ceps['closest'].str.replace('[', '')
df_ceps['closest'] = df_ceps['closest'].str.replace(']', '')
df_ceps['closest'] = df_ceps['closest'].str.replace(' ', '')
df_ceps[['closest1','closest2']] = df_ceps['closest'].str.split(',',expand=True)

df_ceps['closest1'] = df_ceps['closest1'].astype(int)
df_ceps['closest2'] = df_ceps['closest2'].astype(int)

df_ceps.head(10)

,cep,closest,closest1,closest2
0,1000,"10299,11010",10299,11010
1,1001,"10299,11010",10299,11010
2,1002,"10299,11010",10299,11010
3,1003,"10299,11010",10299,11010
4,1004,"10299,11010",10299,11010
5,1005,"10299,11010",10299,11010
6,1006,"10299,11010",10299,11010
7,1007,"10299,11010",10299,11010
8,1008,"10299,11010",10299,11010
9,1009,"10299,11010",10299,11010


In [193]:
# merge da base de ceps com a base cepbr_geo utilizando o cep closest1 como chave
c1 = pd.merge(df_ceps, dataset, left_on=['closest1'], right_on='cep5')
c1.head(10)

,cep_x,closest,closest1,closest2,cep_y,latitude,longitude,cep5
0,1000,"10299,11010",10299,11010,1029901,235427230.0,466332583.0,10299
1,1001,"10299,11010",10299,11010,1029901,235427230.0,466332583.0,10299
2,1002,"10299,11010",10299,11010,1029901,235427230.0,466332583.0,10299
3,1003,"10299,11010",10299,11010,1029901,235427230.0,466332583.0,10299
4,1004,"10299,11010",10299,11010,1029901,235427230.0,466332583.0,10299
5,1005,"10299,11010",10299,11010,1029901,235427230.0,466332583.0,10299
6,1006,"10299,11010",10299,11010,1029901,235427230.0,466332583.0,10299
7,1007,"10299,11010",10299,11010,1029901,235427230.0,466332583.0,10299
8,1008,"10299,11010",10299,11010,1029901,235427230.0,466332583.0,10299
9,1009,"10299,11010",10299,11010,1029901,235427230.0,466332583.0,10299


In [194]:
#merge da base de ceps com a base cepbr_geo utilizando o cep closest2 como chave
c2 = pd.merge(df_ceps, dataset, left_on='closest2', right_on='cep5')
c2.head(10)

,cep_x,closest,closest1,closest2,cep_y,latitude,longitude,cep5
0,1000,"10299,11010",10299,11010,11010120,239344950.0,463244729.0,11010
1,1001,"10299,11010",10299,11010,11010120,239344950.0,463244729.0,11010
2,1002,"10299,11010",10299,11010,11010120,239344950.0,463244729.0,11010
3,1003,"10299,11010",10299,11010,11010120,239344950.0,463244729.0,11010
4,1004,"10299,11010",10299,11010,11010120,239344950.0,463244729.0,11010
5,1005,"10299,11010",10299,11010,11010120,239344950.0,463244729.0,11010
6,1006,"10299,11010",10299,11010,11010120,239344950.0,463244729.0,11010
7,1007,"10299,11010",10299,11010,11010120,239344950.0,463244729.0,11010
8,1008,"10299,11010",10299,11010,11010120,239344950.0,463244729.0,11010
9,1009,"10299,11010",10299,11010,11010120,239344950.0,463244729.0,11010


In [195]:
#concatena as bases c1 e c2, agrupando por cada cep da base de ceps, e cálculando a estimativa de latitude e longitude para cada cep utilizando a média
#entre as latitudes e longitudes dos ceps mais próximos encontrados na base cepbr_geo
df_final = pd.concat((c1,c2)).groupby('cep_x',as_index=False,sort=True)[['latitude', 'longitude']].mean()
df_final.head()

,cep_x,latitude,longitude
0,1000,237386090.0,464788656.0
1,1001,237386090.0,464788656.0
2,1002,237386090.0,464788656.0
3,1003,237386090.0,464788656.0
4,1004,237386090.0,464788656.0


In [196]:
#fortamatação dos dados de latitude e longitude para o formato original
df_final['latitude'] = '-' + df_final['latitude'].astype(int).astype(str)
df_final['longitude'] = '-' + df_final['longitude'].astype(int).astype(str)

df_final.head(10000)

,cep_x,latitude,longitude
0,1000,-237386090,-464788656
1,1001,-237386090,-464788656
2,1002,-237386090,-464788656
3,1003,-237386090,-464788656
4,1004,-237386090,-464788656
...,...,...,...
9995,10995,-239378975,-463259602
9996,10996,-239378975,-463259602
9997,10997,-239378975,-463259602
9998,10998,-239378975,-463259602


In [197]:
#Carga dos dados auxiliares da planilha Desafio2
url = "https://raw.githubusercontent.com/Leticiapp/HandsOnSegfy/main/data/Desafio2.xlsx"
data = rq.get(url).content
df_aux = pd.read_excel(BytesIO(data))

#preenche os campos NAN corretamente
df_aux = df_aux.fillna(method='ffill', axis=0)
df_aux.head(100)

,Estado,Faixa de Cep
0,São Paulo (SP),1000000 a 5999999
1,São Paulo (SP),8000000 a 8499999
2,São Paulo (SP),6000000 a 9999999
3,São Paulo (SP),11000000 a 11999999
4,São Paulo (SP),12000000 a 19999999
...,...,...
89,Santa Catarina (SC),88000000 a 89999999
90,Rio Grande do Sul (RS),90000000 a 91999999
91,Rio Grande do Sul (RS),90000000 a 94900999
92,Rio Grande do Sul (RS),94901000 a 99999999


In [198]:
#Criação dos campos 'min' e 'max', contendo os valores mínimos e máximos do cep de cada região
df_aux['Faixa de Cep'] = df_aux['Faixa de Cep'].str.replace(' ', '')
df_aux[['min','max']] = df_aux['Faixa de Cep'].str.split('a',expand=True).astype(int)

df_aux.head(100)

,Estado,Faixa de Cep,min,max
0,São Paulo (SP),1000000a5999999,1000000,5999999
1,São Paulo (SP),8000000a8499999,8000000,8499999
2,São Paulo (SP),6000000a9999999,6000000,9999999
3,São Paulo (SP),11000000a11999999,11000000,11999999
4,São Paulo (SP),12000000a19999999,12000000,19999999
...,...,...,...,...
89,Santa Catarina (SC),88000000a89999999,88000000,89999999
90,Rio Grande do Sul (RS),90000000a91999999,90000000,91999999
91,Rio Grande do Sul (RS),90000000a94900999,90000000,94900999
92,Rio Grande do Sul (RS),94901000a99999999,94901000,99999999


In [199]:
#mantem apenas as colunas com os maiores de menores valores de cep para cada estado
df_aux1 = pd.concat((df_aux,df_aux)).groupby('Estado',as_index=False)[['min']].min()
df_aux2 = pd.concat((df_aux,df_aux)).groupby('Estado',as_index=False)[['max']].max()

aux = pd.merge(df_aux1, df_aux2, left_on='Estado', right_on='Estado')
aux.head(100)

,Estado,min,max
0,Acre (AC),69900000,69999999
1,Alagoas (AL),57000000,57999999
2,Amapá (AP),68900000,68999999
3,Amazonas (AM),69000000,69899999
4,Bahia (BA),40000000,48999999
5,Ceará,60000000,63999999
6,Distrito Federal (DF),70000000,73699999
7,Espirito Santo (ES),29000000,29999999
8,Goiás (GO),72800000,76799999
9,Maranhão (MA),65000000,65999999


In [200]:
#Criação dos campos min5 e max5, com apenas os 5 primeiros digitos do cep
aux['min'] = aux['min'].astype(str).str.zfill(8)
aux['min5'] = aux['min'].str.slice(0,5).str

aux['max'] = aux['max'].astype(str).str.zfill(8)
aux['max5'] = aux['max'].str.slice(0,5)

#converte para int
aux['min5'] = aux['min5'].astype(int)
aux['max5'] = aux['max5'].astype(int)

aux.head(30)

,Estado,min,max,min5,max5
0,Acre (AC),69900000,69999999,69900,69999
1,Alagoas (AL),57000000,57999999,57000,57999
2,Amapá (AP),68900000,68999999,68900,68999
3,Amazonas (AM),69000000,69899999,69000,69899
4,Bahia (BA),40000000,48999999,40000,48999
5,Ceará,60000000,63999999,60000,63999
6,Distrito Federal (DF),70000000,73699999,70000,73699
7,Espirito Santo (ES),29000000,29999999,29000,29999
8,Goiás (GO),72800000,76799999,72800,76799
9,Maranhão (MA),65000000,65999999,65000,65999


In [201]:
#informações da base auxiliar
aux.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Estado  27 non-null     object
 1   min     27 non-null     object
 2   max     27 non-null     object
 3   min5    27 non-null     int64 
 4   max5    27 non-null     int64 
dtypes: int64(2), object(3)
memory usage: 1.2+ KB


In [202]:
#conversão de tipo de dado
df_final['cep_x'] = df_final['cep_x'].astype(int)

#Criação de um campo estado para nossa base final de ceps
df_final['Estado'] = ''


In [203]:
#Preenchimento do novo campo de estado utilizando a base auxiliar como referência
for i, v in enumerate(df_final['cep_x'].value_counts()):
  for j, v in enumerate(aux['min5'].value_counts()):
        if df_final['cep_x'][i] >= aux['min5'][j] and df_final['cep_x'][i] <= aux['max5'][j]:
            df_final['Estado'][i] = aux['Estado'][j]
            break

In [204]:
df_final.head(20)

,cep_x,latitude,longitude,Estado
0,1000,-237386090,-464788656,São Paulo (SP)
1,1001,-237386090,-464788656,São Paulo (SP)
2,1002,-237386090,-464788656,São Paulo (SP)
3,1003,-237386090,-464788656,São Paulo (SP)
4,1004,-237386090,-464788656,São Paulo (SP)
5,1005,-237386090,-464788656,São Paulo (SP)
6,1006,-237386090,-464788656,São Paulo (SP)
7,1007,-237386090,-464788656,São Paulo (SP)
8,1008,-237386090,-464788656,São Paulo (SP)
9,1009,-237386090,-464788656,São Paulo (SP)


In [205]:
#Renomeia no campo de cep e adiciona os '000' ao final
df_final = df_final.rename(columns={"cep_x": "cep"})
df_final['cep'] = df_final['cep'].astype(str) + '000'


df_final.head(10)

,cep,latitude,longitude,Estado
0,1000000,-237386090,-464788656,São Paulo (SP)
1,1001000,-237386090,-464788656,São Paulo (SP)
2,1002000,-237386090,-464788656,São Paulo (SP)
3,1003000,-237386090,-464788656,São Paulo (SP)
4,1004000,-237386090,-464788656,São Paulo (SP)
5,1005000,-237386090,-464788656,São Paulo (SP)
6,1006000,-237386090,-464788656,São Paulo (SP)
7,1007000,-237386090,-464788656,São Paulo (SP)
8,1008000,-237386090,-464788656,São Paulo (SP)
9,1009000,-237386090,-464788656,São Paulo (SP)


In [206]:
df_final.tail(10)

,cep,latitude,longitude,Estado
98990,99990000,-236978393,-466143895,Rio Grande do Sul (RS)
98991,99991000,-236978393,-466143895,Rio Grande do Sul (RS)
98992,99992000,-236978393,-466143895,Rio Grande do Sul (RS)
98993,99993000,-236978393,-466143895,Rio Grande do Sul (RS)
98994,99994000,-236978393,-466143895,Rio Grande do Sul (RS)
98995,99995000,-236978393,-466143895,Rio Grande do Sul (RS)
98996,99996000,-236978393,-466143895,Rio Grande do Sul (RS)
98997,99997000,-236978393,-466143895,Rio Grande do Sul (RS)
98998,99998000,-236978393,-466143895,Rio Grande do Sul (RS)
98999,99999000,-236978393,-466143895,Rio Grande do Sul (RS)


In [207]:
#Informações do dataset final
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99000 entries, 0 to 98999
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   cep        99000 non-null  object
 1   latitude   99000 non-null  object
 2   longitude  99000 non-null  object
 3   Estado     99000 non-null  object
dtypes: object(4)
memory usage: 3.0+ MB


In [208]:
#Exportação do arquivo final (anexo ao email)
#df_final.to_csv('/content/drive/My Drive/out.csv', index=False)